In [2]:
pip install dask[complete]

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install dask distributed --upgrade

Requirement already up-to-date: dask in /srv/conda/envs/notebook/lib/python3.7/site-packages (2.3.0)
Requirement already up-to-date: distributed in /srv/conda/envs/notebook/lib/python3.7/site-packages (2.3.2)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fastparquet

     |████████████████████████████████| 153kB 3.4MB/s 
     |████████████████████████████████| 3.5MB 54.0MB/s 
     |████████████████████████████████| 61kB 25.6MB/s 
     |████████████████████████████████| 20.2MB 23.6MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.3.2-cp37-cp37m-linux_x86_64.whl size=260880 sha256=5f64226513bb5ce2f82ac92f9895f1984900ce4959645ab9a966fce2885bb482
  Stored in directory: /home/jovyan/.cache/pip/wheels/b9/36/13/01416a760ddcab0eb8281ec9c9ffcbed945c9b831647c8b904
  Created wheel for thrift: filename=thrift-0.11.0-cp37-cp37m-linux_x86_64.whl size=390645 sha256=26d2c103bb10896d8d6cfa9180b992040a76c5800e3e2231e6363bb18a5989d8
  Stored in directory: /home/jovyan/.cache/pip/wheels/be/36/81/0f93ba89a1cb7887c91937948519840a72c0ffdd57cac0ae8f
Successfully built fastparquet thrift
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install gcsfs

     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 102kB 28.5MB/s 
     |████████████████████████████████| 81kB 14.6MB/s 
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
import dask.dataframe as dd
import gcsfs
import fastparquet
from distributed import Client, LocalCluster

In [2]:
#Create Cluster with 10 workers
cluster=LocalCluster(n_workers=10)
client=Client(cluster)

In [3]:
#Load Dataset1
df1=dd.read_parquet(path="gcs://anaconda-public-data/nyc-taxi/nyc.parquet/part.0.parquet", engine='fastparquet')

In [4]:
#Persist Dataset1
df1_clust=client.persist(df1)
df1_clust.head()

,tpep_pickup_datetime,VendorID,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2015-01-01 00:00:00,1,2015-01-01 00:11:26,5,4.00,-73.971436,40.760201,1,N,-73.921181,40.768269,2,13.5,0.5,0.5,0.0,0.0,0.0,14.5
1,2015-01-01 00:00:00,2,2015-01-01 00:00:00,1,1.68,-73.991547,40.750069,1,N,0.000000,0.000000,2,10.0,0.0,0.5,0.0,0.0,0.3,10.8
2,2015-01-01 00:00:00,2,2015-01-01 00:00:00,3,1.56,-74.001320,40.729057,1,N,-74.010208,40.719662,1,7.5,0.5,0.5,0.0,0.0,0.3,8.8
3,2015-01-01 00:00:01,1,2015-01-01 00:03:49,1,0.80,-73.860847,40.757294,1,N,-73.868111,40.752285,2,5.0,0.5,0.5,0.0,0.0,0.0,6.3
4,2015-01-01 00:00:03,2,2015-01-01 00:21:48,2,2.57,-73.969017,40.754269,1,N,-73.994133,40.761600,2,14.5,0.5,0.5,0.0,0.0,0.3,15.8


In [5]:
#Load Dataset2
df2 = pd.read_csv("https://s3.amazonaws.com/nyc-tlc/misc/taxi%20_zone_lookup.csv")
df2.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [6]:
#Extract the hour value from the date time format

def hr_func(ts):
     return ts.hour
    
df1_clust['tpep_pickup_time_hour'] = df1_clust['tpep_pickup_datetime'].apply(hr_func)

/srv/conda/envs/notebook/lib/python3.7/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('tpep_pickup_datetime', 'int64'))

  warnings.warn(meta_warning(meta))


In [7]:
#Group the observations by pickup hour and calculate the total rides taken in each hour
df1_clust_hour = df1_clust.groupby('tpep_pickup_time_hour').count().compute()
df1_clust_hour['tpep_pickup_datetime']

tpep_pickup_time_hour
0     57167
1     54658
2     45702
3     37813
4     26768
5     15379
6     16568
7     19483
8     24440
9     32219
10    35311
11    30869
12    35834
13    36471
14    38254
15    38708
16    34825
17    38191
18    41090
19    39712
20    35170
21    34412
22    35727
23    32698
Name: tpep_pickup_datetime, dtype: int64

In [8]:
#From the above output, peak hours are:
#Midnight-1AM, 1AM-2AM and 2AM-3AM followed by 6PM-7PM
#Peak Hours from 6PM-7PM is logical to understand, howeve peak hour from midnight-3AM doesnt look realistic. We need to look
#into the data to understand this pattern.

#Off-peak hours are from 5AM-6AM

In [9]:
from datetime import datetime

In [10]:
#Calculate the number of rides on 2015-01-01 from midnight-3AM
ret=df1_clust[(df1_clust['tpep_pickup_datetime'] >= '2015-01-01 00:00:00') & (df1_clust['tpep_pickup_datetime'] <= '2015-01-01 02:59:59')]
ret['tpep_pickup_datetime'].count().compute()

88087

In [11]:
#Total rides from Midnight to 3AM = 1,57,527
#Total rides from Midnight to 3AM on 2015-01-01 = 88087
#Conclusion: More than 50% of the total rides in the timeframe of midnight to 3AM are on 2015-01-01 which can be explained due
#to the New Year Eve. The data is highly skewed at midnight on 2015-01-01 causing it to be the peak hours.

#Dataset2 provides the Location ID and corresponding zones, which can be used to determine the peak and off-peak hours in 
#specific zones. However there is no variable linking this dataset to dataset1. Hence it is not used for analysis.